In [1]:
import keras
from keras import layers
from keras.datasets import mnist
from keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt

(x_train, y_train), (x_test, y_test) = mnist.load_data()

2022-11-10 15:55:51.213802: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-10 15:55:51.359228: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-10 15:55:51.363234: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-10 15:55:51.363246: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:
input_vector_size=64
generator = keras.Sequential([
    # Espace latent
    layers.Dense(1568, input_shape=(input_vector_size,)),
    layers.Reshape(target_shape=(7,7,32)),
    layers.Conv2DTranspose(32, kernel_size=(3,3), strides=2, padding="same", activation='relu'),
    layers.Conv2DTranspose(64, kernel_size=(3,3), strides=2, padding="same", activation='relu'),
    layers.Conv2DTranspose(1, kernel_size=(3,3), strides=1, padding="same", activation='relu')
])
generator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1568)              101920    
                                                                 
 reshape (Reshape)           (None, 7, 7, 32)          0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 14, 14, 32)       9248      
 nspose)                                                         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 28, 28, 64)       18496     
 ranspose)                                                       
                                                                 
 conv2d_transpose_2 (Conv2DT  (None, 28, 28, 1)        577       
 ranspose)                                                       
                                                        

2022-11-10 15:55:53.314437: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-10 15:55:53.314457: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-10 15:55:53.314471: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mithrandir-laptop): /proc/driver/nvidia/version does not exist
2022-11-10 15:55:53.314664: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
discriminator = keras.Sequential([
    layers.Input(shape=(28,28,1)),
    layers.Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(16, kernel_size=(3, 3), padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(1, activation='sigmoid')
])
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 128)       1280      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 128)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        73792     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 32)          18464     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 3, 3, 32)        

In [4]:
generator.compile(loss="binary_crossentropy", optimizer="adam")
discriminator.compile(loss="binary_crossentropy", optimizer="adam")

In [5]:
discriminator.trainable=False

z = keras.layers.Input(shape=(input_vector_size,))
image = generator(z)

valid = discriminator(image)

combined_network = keras.Model(z,valid)
combined_network.compile(loss='binary_crossentropy',optimizer=Adam(0.001))



In [6]:
def display_images():
        r, c = 4,4
        noise = np.random.normal(0, 1, (r * c,input_vector_size))
        generated_images = generator.predict(noise)

        #Scaling the generated images
        generated_images = 0.5 * generated_images + 0.5

        fig, axs = plt.subplots(r, c)
        count = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(generated_images[count, :,:,])
                axs[i,j].axis('off')
                count += 1
        plt.show()
        plt.close()

In [ ]:
num_epochs = 10000
batch_size = 32

valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size,1))

display_interval = 100
valid = 0.05 * np.random.random(valid.shape)
fake = np.zeros((batch_size, 1))
fake += 0.05 * np.random.random(fake.shape)
for epoch in range(num_epochs):


    discm_loss_real = 0
    discm_loss_fake = 0
    genr_loss = 0
    for i in range(np.random.randint(1,10)):
        index = np.random.randint(0, x_train.shape[0], batch_size)
        images = x_train[index]

        noise = np.random.normal(0,1,(batch_size, input_vector_size))
        generated_images = generator.predict(noise)
        discm_loss_real = discriminator.train_on_batch(images, valid)
        discm_loss_fake = discriminator.train_on_batch(generated_images, fake)

        genr_loss=combined_network.train_on_batch(noise,valid)



    if epoch % display_interval == 0:
        print("EPOCH === ", epoch)
        print('Generator loss = ', genr_loss)
        print('Discriminator_loss = ', discm_loss_fake)
        display_images()


1/1 [==============================] - 0s 28ms/step
